In [4]:
import spacy
from pattern.en import conjugate, tenses

nlp = spacy.load('en_core_web_sm')

noundict = {'i':'me', 'we':'us', 'you':'you', 'he':'him', 'she':'her', 'they':'them', 'them':'they', 'her':'she', 'him':'he', 'us':'we', 'me':'i'}

def nouninv(noun):
    n = noun.lower()
    if n in noundict:
        return noundict[n]
    return noun

def pass2act(doc, rec=False):
    parse = nlp(doc)
    newdoc = ''
    for sent in parse.sents:

        # Init parts of sentence to capture:
        subjpass = ''
        subj = ''
        verb = ''
        verbaspect = ''
        verbtense = ''
        adverb = {'bef':'', 'aft':''}
        part = ''
        prep = ''
        agent = ''
        aplural = False
        advcltree = None
        aux = []
        xcomp = ''
        punc = '.'
        # Analyse dependency tree:
        for word in sent:
            if word.dep_ == 'advcl':
                if word.head.dep_ in ('ROOT', 'auxpass'):
                    advcltree = word.subtree
            if word.dep_ == 'nsubjpass':
                if word.head.dep_ == 'ROOT':
                    subjpass = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'nsubj':
                subj = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                if word.head.dep_ == 'auxpass':
                    if word.head.head.dep_ == 'ROOT':
                        subjpass = subj
            if word.dep_ in ('advmod','npadvmod','oprd'):
                if word.head.dep_ == 'ROOT':
                    if verb == '':
                        adverb['bef'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    else:
                        adverb['aft'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'auxpass':
                if word.head.dep_ == 'ROOT':
                    if not subjpass:
                        subjpass = subj
            if word.dep_ in ('aux','auxpass','neg'):
                if word.head.dep_ == 'ROOT':
                    aux.append(word)
            if word.dep_ == 'ROOT':
                verb = word.text
                if word.tag_ == 'VB':
                    verbtense = 'inf'
                elif word.tag_ == 'VBD':
                    verbtense = 'past'
                elif word.tag_ == 'VBG':
                    verbtense = 'pres'
                    verbaspect = 'progressive'
                elif word.tag_ == 'VBN':
                    verbtense = 'past'
                else:
                    verbtense = tenses(word.text)[0][0]
            if word.dep_ == 'prt':
                if word.head.dep_ == 'ROOT':
                    part = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'prep':
                if word.head.dep_ == 'ROOT':
                    prep = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_.endswith('obj'):
                if word.head.dep_ == 'agent':
                    if word.head.head.dep_ == 'ROOT':
                        agent = ''.join(w.text + ', ' if w.dep_=='appos' else (w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws) for w in word.subtree).strip()
                        aplural = word.tag_ in ('NNS','NNPS')
            if word.dep_ in ('xcomp','ccomp','conj'):
                if word.head.dep_ == 'ROOT':
                    xcomp = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    that = xcomp.startswith('that')
                    xcomp = pass2act(xcomp, True).strip(' .')
                    if not xcomp.startswith('that') and that:
                        xcomp = 'that '+xcomp
            if word.dep_ == 'punct' and not rec:
                if word.text != '"':
                    punc = word.text

        # exit if not passive:
        if subjpass == '':
            newdoc += str(sent) + ' '
            continue

        # if no agent is found:
        if agent == '':
            # what am I gonna do? BITconEEEEEEECT!!!!
            newdoc += str(sent) + ' '
            continue

        # invert nouns:
        agent = nouninv(agent)
        subjpass = nouninv(subjpass)

        # CONJUGATION
        auxstr = ''
        num = 'sg' if not aplural or agent in ('he','she') else 'pl'
        aux.append(aux[0])
        verbaspect = None
        for (pp, p, a, n) in zip(aux,aux[1:],aux[2:],aux[3:]):
            if a.lemma_ == '.':
                continue

            if a.lemma_ == 'not':
                if p.lemma_ == 'be':
                    if n.lemma_ == 'be':
                        verbtense = tenses(a.text)[0][0]
                        auxstr += conjugate('be',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbaspect = 'progressive'
                    else:
                        auxstr += conjugate('do',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbtense = 'inf'
                auxstr += 'not '
            elif a.lemma_ == 'be':
                if p.lemma_ == 'be':
                    verbtense = tenses(a.text)[0][0]
                    auxstr += conjugate('be',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                    verbaspect = 'progressive'
                elif p.tag_ == 'MD':
                    verbtense = 'inf'
            elif a.lemma_ == 'have':
                num == 'pl' if p.tag_ == 'MD' else num
                auxstr += conjugate('have',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                if n.lemma_ == 'be':
                    verbaspect = 'progressive'
                    verbtense = tenses(n.text)[0][0]
            else:
                auxstr += a.text_with_ws
        auxstr = auxstr.lower().strip()

        if verbaspect:
            verb = conjugate(verb,tense=verbtense,aspect=verbaspect)
        else:
            verb = conjugate(verb,tense=verbtense)

        advcl = ''
        if advcltree:
            for w in advcltree:
                if w.pos_ == 'VERB' and tenses(w.text)[0][4] == 'progressive':
                    advcl += 'which ' + conjugate(w.text,tense=tenses(verb)[0][0]) + ' '
                else:
                    advcl += w.text_with_ws

        newsent = ' '.join(list(filter(None, [agent,auxstr,adverb['bef'],verb,part,subjpass,adverb['aft'],advcl,prep,xcomp])))+punc
        if not rec:
            newsent = newsent[0].upper() + newsent[1:]
        newdoc += newsent + ' '
    return newdoc

# Example usage:
sentence = "ram painted the house"
converted_sentence = pass2act(sentence)
print(converted_sentence)


ram painted the house 


In [25]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

def convert_to_passive(sentence):
    # Tokenize the sentence
    tokens = word_tokenize(sentence)

    # Perform part-of-speech tagging
    tagged = pos_tag(tokens)

    # Convert Active Voice to Passive Voice
    # Remove the first tag from the list if it's 'PRP' or 'NN'
    if tagged[0][1] == 'PRP' or tagged[0][1] == 'NN':
        tags = tagged[1:]
    else:
        tags = tagged

    verbs = [item[0] for item in tags if item[1].startswith('V')][0]

    # Lemmatize the verb
    lemmatizer = WordNetLemmatizer()
    verbs = lemmatizer.lemmatize(verbs, pos='v')

    new_sentence = ""
    # If the first item in tags is 'IN', add it to the new sentence
    if tags[0][1] == 'IN':
        new_sentence = new_sentence + tags[0][0] + " "

    # Get the first 'NN' from tags list and add it to the new sentence
    for item in tags:
        if item[1] == 'NN':
            new_sentence += item[0]
            # Remove the item from tags list
            tags.remove(item)
            break

    # Get 'IN' from tags list and add it to the new sentence
    for item in tags:
        if item[1] == 'IN':
            if item[0] not in new_sentence:
                new_sentence += " " + item[0]
                # Remove the item from tags list
                tags.remove(item)
            break

    # Get the second 'NN' from tags list and add it to the new sentence
    for item in tags:
        if item[1] == 'NN':
            new_sentence += " " + item[0]
            # Remove the item from tags list
            tags.remove(item)
            break

    new_sentence = new_sentence + " " + verbs

    return new_sentence

# Example usage:
active_sentence = "The author wrote the book."
passive_sentence = convert_to_passive(active_sentence)
print("Active sentence:", active_sentence)
print("Passive sentence:", passive_sentence)

Active sentence: The author wrote the book.
Passive sentence: author book write


In [5]:
import spacy
from pattern.en import conjugate, tenses

nlp = spacy.load('en_core_web_sm')

noundict = {'i':'me', 'we':'us', 'you':'you', 'he':'him', 'she':'her', 'they':'them', 'them':'they', 'her':'she', 'him':'he', 'us':'we', 'me':'i'}

def nouninv(noun):
    n = noun.lower()
    if n in noundict:
        return noundict[n]
    return noun

def pass2act(doc, rec=False):
    parse = nlp(doc)
    newdoc = ''
    for sent in parse.sents:

        # Init parts of sentence to capture:
        subjpass = ''
        subj = ''
        verb = ''
        verbaspect = ''
        verbtense = ''
        adverb = {'bef':'', 'aft':''}
        part = ''
        prep = ''
        agent = ''
        aplural = False
        advcltree = None
        aux = []
        xcomp = ''
        punc = '.'
        # Analyse dependency tree:
        for word in sent:
            if word.dep_ == 'advcl':
                if word.head.dep_ in ('ROOT', 'auxpass'):
                    advcltree = word.subtree
            if word.dep_ == 'nsubjpass':
                if word.head.dep_ == 'ROOT':
                    subjpass = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'nsubj':
                subj = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                if word.head.dep_ == 'auxpass':
                    if word.head.head.dep_ == 'ROOT':
                        subjpass = subj
            if word.dep_ in ('advmod','npadvmod','oprd'):
                if word.head.dep_ == 'ROOT':
                    if verb == '':
                        adverb['bef'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    else:
                        adverb['aft'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'auxpass':
                if word.head.dep_ == 'ROOT':
                    if not subjpass:
                        subjpass = subj
            if word.dep_ in ('aux','auxpass','neg'):
                if word.head.dep_ == 'ROOT':
                    aux.append(word)
            if word.dep_ == 'ROOT':
                verb = word.text
                if word.tag_ == 'VB':
                    verbtense = 'inf'
                elif word.tag_ == 'VBD':
                    verbtense = 'past'
                elif word.tag_ == 'VBG':
                    verbtense = 'pres'
                    verbaspect = 'progressive'
                elif word.tag_ == 'VBN':
                    verbtense = 'past'
                else:
                    verbtense = tenses(word.text)[0][0]
            if word.dep_ == 'prt':
                if word.head.dep_ == 'ROOT':
                    part = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'prep':
                if word.head.dep_ == 'ROOT':
                    prep = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_.endswith('obj'):
                if word.head.dep_ == 'agent':
                    if word.head.head.dep_ == 'ROOT':
                        agent = ''.join(w.text + ', ' if w.dep_=='appos' else (w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws) for w in word.subtree).strip()
                        aplural = word.tag_ in ('NNS','NNPS')
            if word.dep_ in ('xcomp','ccomp','conj'):
                if word.head.dep_ == 'ROOT':
                    xcomp = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    that = xcomp.startswith('that')
                    xcomp = pass2act(xcomp, True).strip(' .')
                    if not xcomp.startswith('that') and that:
                        xcomp = 'that '+xcomp
            if word.dep_ == 'punct' and not rec:
                if word.text != '"':
                    punc = word.text

        # exit if not passive:
        if subjpass == '':
            newdoc += str(sent) + ' '
            continue

        # if no agent is found:
        if agent == '':
            # what am I gonna do? BITconEEEEEEECT!!!!
            newdoc += str(sent) + ' '
            continue

        # invert nouns:
        agent = nouninv(agent)
        subjpass = nouninv(subjpass)

        # CONJUGATION
        auxstr = ''
        num = 'sg' if not aplural or agent in ('he','she') else 'pl'
        aux.append(aux[0])
        verbaspect = None
        for (pp, p, a, n) in zip(aux,aux[1:],aux[2:],aux[3:]):
            if a.lemma_ == '.':
                continue

            if a.lemma_ == 'not':
                if p.lemma_ == 'be':
                    if n.lemma_ == 'be':
                        verbtense = tenses(a.text)[0][0]
                        auxstr += conjugate('be',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbaspect = 'progressive'
                    else:
                        auxstr += conjugate('do',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbtense = 'inf'
                auxstr += 'not '
            elif a.lemma_ == 'be':
                if p.lemma_ == 'be':
                    verbtense = tenses(a.text)[0][0]
                    auxstr += conjugate('be',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                    verbaspect = 'progressive'
                elif p.tag_ == 'MD':
                    verbtense = 'inf'
            elif a.lemma_ == 'have':
                num == 'pl' if p.tag_ == 'MD' else num
                auxstr += conjugate('have',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                if n.lemma_ == 'be':
                    verbaspect = 'progressive'
                    verbtense = tenses(n.text)[0][0]
            else:
                auxstr += a.text_with_ws
        auxstr = auxstr.lower().strip()

        if verbaspect:
            verb = conjugate(verb,tense=verbtense,aspect=verbaspect)
        else:
            verb = conjugate(verb,tense=verbtense)

        advcl = ''
        if advcltree:
            for w in advcltree:
                if w.pos_ == 'VERB' and tenses(w.text)[0][4] == 'progressive':
                    advcl += 'which ' + conjugate(w.text,tense=tenses(verb)[0][0]) + ' '
                else:
                    advcl += w.text_with_ws

        newsent = ' '.join(list(filter(None, [agent,auxstr,adverb['bef'],verb,part,subjpass,adverb['aft'],advcl,prep,xcomp])))+punc
        if not rec:
            newsent = newsent[0].upper() + newsent[1:]
        newdoc += newsent + ' '
    return newdoc

# Example usage:
sentence = "The book was written by the author."
converted_sentence = pass2act(sentence)
print(converted_sentence)

RuntimeError: generator raised StopIteration

In [6]:
import spacy
from pattern.en import conjugate, tenses

nlp = spacy.load('en_core_web_sm')

noundict = {'i':'me', 'we':'us', 'you':'you', 'he':'him', 'she':'her', 'they':'them', 'them':'they', 'her':'she', 'him':'he', 'us':'we', 'me':'i'}

def nouninv(noun):
    n = noun.lower()
    if n in noundict:
        return noundict[n]
    return noun

def pass2act(doc, rec=False):
    parse = nlp(doc)
    newdoc = ''
    for sent in parse.sents:

        # Init parts of sentence to capture:
        subjpass = ''
        subj = ''
        verb = ''
        verbaspect = ''
        verbtense = ''
        adverb = {'bef':'', 'aft':''}
        part = ''
        prep = ''
        agent = ''
        aplural = False
        advcltree = None
        aux = []
        xcomp = ''
        punc = '.'
        # Analyse dependency tree:
        for word in sent:
            if word.dep_ == 'advcl':
                if word.head.dep_ in ('ROOT', 'auxpass'):
                    advcltree = word.subtree
            if word.dep_ == 'nsubjpass':
                if word.head.dep_ == 'ROOT':
                    subjpass = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'nsubj':
                subj = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                if word.head.dep_ == 'auxpass':
                    if word.head.head.dep_ == 'ROOT':
                        subjpass = subj
            if word.dep_ in ('advmod','npadvmod','oprd'):
                if word.head.dep_ == 'ROOT':
                    if verb == '':
                        adverb['bef'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    else:
                        adverb['aft'] = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'auxpass':
                if word.head.dep_ == 'ROOT':
                    if not subjpass:
                        subjpass = subj
            if word.dep_ in ('aux','auxpass','neg'):
                if word.head.dep_ == 'ROOT':
                    aux.append(word)
            if word.dep_ == 'ROOT':
                verb = word.text
                if word.tag_ == 'VB':
                    verbtense = 'inf'
                elif word.tag_ == 'VBD':
                    verbtense = 'past'
                elif word.tag_ == 'VBG':
                    verbtense = 'pres'
                    verbaspect = 'progressive'
                elif word.tag_ == 'VBN':
                    verbtense = 'past'
                else:
                    verbtense = tenses(word.text)[0][0]
            if word.dep_ == 'prt':
                if word.head.dep_ == 'ROOT':
                    part = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_ == 'prep':
                if word.head.dep_ == 'ROOT':
                    prep = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
            if word.dep_.endswith('obj'):
                if word.head.dep_ == 'agent':
                    if word.head.head.dep_ == 'ROOT':
                        agent = ''.join(w.text + ', ' if w.dep_=='appos' else (w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws) for w in word.subtree).strip()
                        aplural = word.tag_ in ('NNS','NNPS')
            if word.dep_ in ('xcomp','ccomp','conj'):
                if word.head.dep_ == 'ROOT':
                    xcomp = ''.join(w.text_with_ws.lower() if w.tag_ not in ('NNP','NNPS') else w.text_with_ws for w in word.subtree).strip()
                    that = xcomp.startswith('that')
                    xcomp = pass2act(xcomp, True).strip(' .')
                    if not xcomp.startswith('that') and that:
                        xcomp = 'that '+xcomp
            if word.dep_ == 'punct' and not rec:
                if word.text != '"':
                    punc = word.text

        # exit if not passive:
        if subjpass == '':
            newdoc += str(sent) + ' '
            continue

        # if no agent is found:
        if agent == '':
            # what am I gonna do? BITconEEEEEEECT!!!!
            newdoc += str(sent) + ' '
            continue

        # invert nouns:
        agent = nouninv(agent)
        subjpass = nouninv(subjpass)

        # CONJUGATION
        auxstr = ''
        num = 'sg' if not aplural or agent in ('he','she') else 'pl'
        aux.append(aux[0])
        verbaspect = None
        for (pp, p, a, n) in zip(aux,aux[1:],aux[2:],aux[3:]):
            if a.lemma_ == '.':
                continue

            if a.lemma_ == 'not':
                if p.lemma_ == 'be':
                    if n.lemma_ == 'be':
                        verbtense = tenses(a.text)[0][0]
                        auxstr += conjugate('be',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbaspect = 'progressive'
                    else:
                        auxstr += conjugate('do',tense=tenses(p.text)[0][0],person=3,number=num) + ' '
                        verbtense = 'inf'
                auxstr += 'not '
            elif a.lemma_ == 'be':
                if p.lemma_ == 'be':
                    verbtense = tenses(a.text)[0][0]
                    auxstr += conjugate('be',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                    verbaspect = 'progressive'
                elif p.tag_ == 'MD':
                    verbtense = 'inf'
            elif a.lemma_ == 'have':
                num == 'pl' if p.tag_ == 'MD' else num
                auxstr += conjugate('have',tense=tenses(a.text)[0][0],person=3,number=num) + ' '
                if n.lemma_ == 'be':
                    verbaspect = 'progressive'
                    verbtense = tenses(n.text)[0][0]
            else:
                auxstr += a.text_with_ws
        auxstr = auxstr.lower().strip()

        if verbaspect:
            verb = conjugate(verb,tense=verbtense,aspect=verbaspect)
        else:
            verb = conjugate(verb,tense=verbtense)

        advcl = ''
        if advcltree:
            for w in advcltree:
                if w.pos_ == 'VERB' and tenses(w.text)[0][4] == 'progressive':
                    advcl += 'which ' + conjugate(w.text,tense=tenses(verb)[0][0]) + ' '
                else:
                    advcl += w.text_with_ws

        newsent = ' '.join(list(filter(None, [agent,auxstr,adverb['bef'],verb,part,subjpass,adverb['aft'],advcl,prep,xcomp])))+punc
        if not rec:
            newsent = newsent[0].upper() + newsent[1:]
        newdoc += newsent + ' '
    return newdoc

# Example usage:
sentence = "The cake was baked by the chef."
converted_sentence = pass2act(sentence)
print(converted_sentence)


The chef baked the cake. 


In [7]:
sentence1 = "The book was written by the author."
sentence2 = "The cake was baked by Mary."
sentence3 = "The letter was mailed by John."

converted_sentence1 = pass2act(sentence1)
converted_sentence2 = pass2act(sentence2)
converted_sentence3 = pass2act(sentence3)

print("Original sentence 1:", sentence1)
print("Converted sentence 1:", converted_sentence1)

print("Original sentence 2:", sentence2)
print("Converted sentence 2:", converted_sentence2)

print("Original sentence 3:", sentence3)
print("Converted sentence 3:", converted_sentence3)


Original sentence 1: The book was written by the author.
Converted sentence 1: The author wrote the book. 
Original sentence 2: The cake was baked by Mary.
Converted sentence 2: Mary baked the cake. 
Original sentence 3: The letter was mailed by John.
Converted sentence 3: John mailed the letter. 


In [22]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [17]:
from hazm import (
    Normalizer, word_tokenize, POSTagger,
    Chunker, tree2brackets, Lemmatizer,
    DependencyParser, Stemmer
)

In [ ]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()
sentence = "The author wrote the book"
# normalize
normalized_sentence = normalizer.normalize(sentence)
print(normalized_sentence)

In [ ]:
tokens = word_tokenize(normalized_sentence)
print(tokens)
tagger = POSTagger(model='resources/postagger.model')
tagged = tagger.tag(tokens)
print(tagged)

In [ ]:
chunker = Chunker(model='resources/chunker.model')
print(tree2brackets(chunker.parse(tagged)))

parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)
parser.parse(tokens)

In [ ]:
print(tagged)
# Convert Active Voice to Passive Voice
# remove first tag from list if its 'PRO' or 'N'
if tagged[0][1] == 'PRO' or tagged[0][1] == 'N':
    tags = tagged[1:]
else:
    tags = tagged
print(tags)

In [ ]:
verbs = [item[0] for item in tags if item[1] == 'V'][0]
print(verbs)
verbs = lemmatizer.lemmatize(verbs).split("#")
print(verbs)

In [ ]:
new_sentence = ""
# if first item in tags is 'P' add it to new sentence
if tags[0][1] == 'P':
    new_sentence = new_sentence + tags[0][0] + " "
# get first 'N' from tags list and add it to new sentence
for item in tags:
    if item[1] == 'N':
        new_sentence += lemmatizer.lemmatize(item[0])
        # remote item from tags list
        tags.remove(item)
        break

# get 'P' from tags list and add it to new sentence
for item in tags:
    if item[1] == 'P':
        if item[0] not in new_sentence:
            new_sentence += " " + item[0]
            # remote item from tags list
            tags.remove(item)
        break

# get second 'N' from tags list and add it to new sentence
for item in tags:
    if item[1] == 'N':
        new_sentence += " " + item[0]
        # remote item from tags list
        tags.remove(item)
        break

new_sentence = new_sentence + " " + verbs[0] + "ه" + " " + "شد."
print("Active sentence: ", sentence)
print("Passive sentence: ", new_sentence)

In [16]:
pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 22.6 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=c5ec117284d1859086e01408f6dc9bdc048a0a8c6ff0dd4126c09b250a08910c
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [8]:
pip install textblob

In [2]:
pip install pattern

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332702 sha256=e3c0e79b61d661028115148d54a347b263ef789ca3cb2b259b2243d8b630ac68
  Stored in dire

In [28]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

active_rule = [{'DEP': {"IN": ['nsubj', 'aux']}}]

matcher = Matcher(nlp.vocab)
matcher.add("Active_rule", [active_rule])

def is_active(sentence):
    doc = nlp(sentence)
    matches = matcher(doc)

    if len(matches) > 0:
        return True
    else:
        return False

# Example usage:
sentence1 = "The book was written by the author."
sentence2 = "The author wrote the book."
sentence3 = "The dog is cased by the cat"

print("Is sentence1 active?", is_active(sentence1))
print("Is sentence2 active?", is_active(sentence2))
print("Is sentence3 active?", is_active(sentence3))

Is sentence1 active? False
Is sentence2 active? True
Is sentence3 active? True


In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

passive_rule = [{'DEP': {"IN": ['nsubjpass', 'auxpass']}}]

matcher = Matcher(nlp.vocab)
matcher.add("Passive_rule", [passive_rule])

def is_passive(sentence):
    doc = nlp(sentence)
    matches = matcher(doc)

    if len(matches) > 0:
        return True
    else:
        return False

# Example usage:
sentence1 = "The book was written by the author."
sentence2 = "The author wrote the book."

print("Is sentence1 passive?", is_passive(sentence1))
print("Is sentence2 passive?", is_passive(sentence2))